# Cgroups

**cgroups** (abbreviated from control groups) is a Linux kernel feature that limits, accounts for, and isolates the resource usage (CPU, memory, disk I/O, network, etc.) of a collection of processes.

A control group is a collection of processes that are bound by the same criteria and associated with a set of parameters or limits. These groups can be hierarchical, meaning that each group inherits limits from its parent group. The kernel provides access to multiple controllers (also called subsystems) through the cgroup interface, for example, the "memory" controller limits memory use, "cpuacct" accounts CPU usage, etc.

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

2017-08-09 15:04:59,924 INFO    : root         : Using LISA logging configuration:
2017-08-09 15:04:59,926 INFO    : root         :   /media/build/workspace/lisa-github/logging.conf


In [2]:
import os
import json
import operator

import devlib
import trappy
import bart

from bart.sched.SchedMultiAssert import SchedMultiAssert
from wlgen import RTA, Periodic

## Target Configuration
The target configuration is used to describe and configure your test environment.
You can find more details in **examples/utils/testenv_example.ipynb**.

In [3]:
from env import TestEnv

my_conf = {

    "platform"    : 'android',
    "board"       : "hikey960",
    "device" : "0123456789ABCDEF",
    
    "ANDROID_HOME" : "/media/build/workspace/android-sdk",
    "rtapp-calib" : {"0": 302, "1": 302, "2": 304, "3": 304, "4": 136, "5": 137, "6": 136, "7": 136},

     "emeter" : {
        "instrument" : "acme",
        "conf" : {
            # Absolute path to the iio-capture binary on the host
            'iio-capture' : '/usr/bin/iio-capture',
            # Default host name of the BeagleBone Black
            'ip_address'     : 'baylibre-acme.local',
        },
        "channel_map" : {
            "Device0" : 0, # iio:device0
        }
    },

    
    "exclude_modules" : [ "hwmon" ],

    # List of additional devlib modules to install 
    "modules" : ['cgroups', 'bl', 'cpufreq'],
    
    # List of additional binary tools to install
    "tools" : ['rt-app', 'trace-cmd'],
    
    # FTrace events to collect
    "ftrace" : {
         "events" : [
             "sched_switch"
         ],
         "buffsize" : 10240
    }
}

te = TestEnv(my_conf, force_new=True)
target = te.target

# Report target connection
logging.info('Connected to %s target', target.abi)
print "DONE"

2017-08-09 15:05:02,330 INFO    : TestEnv      : Using base path: /media/build/workspace/lisa-github
2017-08-09 15:05:02,343 INFO    : TestEnv      : Loading custom (inline) target configuration
2017-08-09 15:05:02,346 INFO    : TestEnv      : External tools using:
2017-08-09 15:05:02,348 INFO    : TestEnv      :    ANDROID_HOME: /media/build/workspace/android-sdk
2017-08-09 15:05:02,352 INFO    : TestEnv      :    CATAPULT_HOME: /media/build/workspace/lisa-github/tools/catapult
2017-08-09 15:05:02,353 INFO    : TestEnv      : Devlib modules to load: ['bl', 'cpuidle', 'cpufreq', 'cgroups']
2017-08-09 15:05:02,353 INFO    : TestEnv      : Connecting Android target [0123456789ABCDEF]
2017-08-09 15:05:02,354 INFO    : TestEnv      : Connection settings:
2017-08-09 15:05:02,355 INFO    : TestEnv      :    {'device': '0123456789ABCDEF'}
2017-08-09 15:05:03,454 INFO    : TestEnv      : Initializing target workdir:
2017-08-09 15:05:03,457 INFO    : TestEnv      :    /data/local/tmp/devlib-tar

DONE


## List available Controllers

Details on the available controllers (or subsystems) can be found at: https://www.kernel.org/doc/Documentation/cgroup-v1/.

In [4]:
logging.info('%14s - Available controllers:', 'CGroup')
ssys = target.cgroups.list_subsystems()
for (n,h,g,e) in ssys:
    logging.info('%14s -    %10s (hierarchy id: %d) has %d cgroups',
                 'CGroup', n, h, g)

2017-08-09 15:05:23,493 INFO    : root         :         CGroup - Available controllers:
2017-08-09 15:05:23,592 INFO    : root         :         CGroup -        cpuset (hierarchy id: 5) has 8 cgroups
2017-08-09 15:05:23,593 INFO    : root         :         CGroup -           cpu (hierarchy id: 4) has 1 cgroups
2017-08-09 15:05:23,596 INFO    : root         :         CGroup -       cpuacct (hierarchy id: 1) has 55 cgroups
2017-08-09 15:05:23,597 INFO    : root         :         CGroup -     schedtune (hierarchy id: 2) has 4 cgroups
2017-08-09 15:05:23,598 INFO    : root         :         CGroup -         blkio (hierarchy id: 6) has 1 cgroups
2017-08-09 15:05:23,598 INFO    : root         :         CGroup -        memory (hierarchy id: 3) has 40 cgroups
2017-08-09 15:05:23,599 INFO    : root         :         CGroup -       devices (hierarchy id: 6) has 1 cgroups
2017-08-09 15:05:23,604 INFO    : root         :         CGroup -       freezer (hierarchy id: 6) has 1 cgroups
2017-08-09 15

## Example of CPUSET controller usage

Cpusets provide a mechanism for assigning a set of CPUs and memory nodes to a set of tasks. Cpusets constrain the CPU and memory placement of tasks to only the resources available within a task's current cpuset.  They form a nested hierarchy visible in a virtual file system.  These are the essential hooks, beyond what is already present, required to manage dynamic job placement on large systems.

More information can be found in the kernel documentation: https://www.kernel.org/doc/Documentation/cgroup-v1/cpusets.txt.

In [5]:
# Get a reference to the CPUSet controller
cpuset = target.cgroups.controller('cpuset')

In [6]:
# Get the list of current configured CGroups for that controller
cgroups = cpuset.list_all()
logging.info('Existing CGropups:')
for cg in cgroups:
    logging.info('  %s', cg)

2017-08-09 15:06:44,962 INFO    : root         : Existing CGropups:
2017-08-09 15:06:44,963 INFO    : root         :   /
2017-08-09 15:06:44,964 INFO    : root         :   /big
2017-08-09 15:06:44,964 INFO    : root         :   /LITTLE
2017-08-09 15:06:44,965 INFO    : root         :   /system-background
2017-08-09 15:06:44,966 INFO    : root         :   /background
2017-08-09 15:06:44,967 INFO    : root         :   /foreground
2017-08-09 15:06:44,968 INFO    : root         :   /foreground/boost
2017-08-09 15:06:44,969 INFO    : root         :   /top-app


In [7]:
# Dump the configuraiton of each controller
for cgname in cgroups:
    #print cgname
    cgroup = cpuset.cgroup(cgname)
    attrs = cgroup.get()
    #print attrs
    cpus = attrs['cpus']
    logging.info('%s:%-15s cpus: %s', cpuset.kind, cgroup.name, cpus)
    

2017-08-09 15:07:02,049 INFO    : root         : cpuset:/               cpus: 0-7
2017-08-09 15:07:02,265 INFO    : root         : cpuset:/big            cpus: 4-7
2017-08-09 15:07:02,480 INFO    : root         : cpuset:/LITTLE         cpus: 0-3
2017-08-09 15:07:02,670 INFO    : root         : cpuset:/system-background cpus: 0-3
2017-08-09 15:07:02,854 INFO    : root         : cpuset:/background     cpus: 0
2017-08-09 15:07:03,061 INFO    : root         : cpuset:/foreground     cpus: 0-7
2017-08-09 15:07:03,298 INFO    : root         : cpuset:/foreground/boost cpus: 0-7
2017-08-09 15:07:03,511 INFO    : root         : cpuset:/top-app        cpus: 0-7


In [8]:
# Create a LITTLE partition
cpuset_littles = cpuset.cgroup('/LITTLE')

In [9]:
# Check the attributes available for this control group
print "LITTLE:\n", json.dumps(cpuset_littles.get(), indent=4)

LITTLE:
{
    "memory_pressure": "0", 
    "memory_spread_page": "0", 
    "notify_on_release": "0", 
    "sched_load_balance": "1", 
    "cpus": "0-3", 
    "effective_mems": "0", 
    "memory_spread_slab": "0", 
    "mem_hardwall": "0", 
    "cpu_exclusive": "0", 
    "mem_exclusive": "0", 
    "ls": " /data/local/tmp/devlib-target/cgroups/devlib_cgh5/LITTLE/cpuset.*", 
    "mems": "0", 
    "memory_migrate": "0", 
    "sched_relax_domain_level": "-1", 
    "effective_cpus": "0-3"
}


In [10]:
# Tune CPUs and MEMs attributes
#   they must be initialize for the group to be usable
cpuset_littles.set(cpus=target.bl.littles, mems=0)
print "LITTLE:\n", json.dumps(cpuset_littles.get(), indent=4)

LITTLE:
{
    "memory_pressure": "0", 
    "memory_spread_page": "0", 
    "notify_on_release": "0", 
    "sched_load_balance": "1", 
    "cpus": "0-3", 
    "effective_mems": "0", 
    "memory_spread_slab": "0", 
    "mem_hardwall": "0", 
    "cpu_exclusive": "0", 
    "mem_exclusive": "0", 
    "ls": " /data/local/tmp/devlib-target/cgroups/devlib_cgh5/LITTLE/cpuset.*", 
    "mems": "0", 
    "memory_migrate": "0", 
    "sched_relax_domain_level": "-1", 
    "effective_cpus": "0-3"
}


In [11]:
# Define a periodic big (80%) task
task = Periodic(
    period_ms=100,
    duty_cycle_pct=80,
    duration_s=5).get()

# Create one task per each CPU in the target
tasks={}
for tid in enumerate(target.core_names):
    tasks['task{}'.format(tid[0])] = task

# Configure RTA to run all these tasks
rtapp = RTA(target, 'simple', calibration=te.calibration())
rtapp.conf(kind='profile', params=tasks, run_dir=target.working_directory);

2017-08-09 15:08:17,118 INFO    : Workload     : Setup new workload simple
2017-08-09 15:08:17,206 INFO    : Workload     : Workload duration defined by longest task
2017-08-09 15:08:17,207 INFO    : Workload     : Default policy: SCHED_OTHER
2017-08-09 15:08:17,210 INFO    : Workload     : ------------------------
2017-08-09 15:08:17,211 INFO    : Workload     : task [task0], sched: using default policy
2017-08-09 15:08:17,213 INFO    : Workload     :  | loops count: 1
2017-08-09 15:08:17,215 INFO    : Workload     : + phase_000001: duration 5.000000 [s] (50 loops)
2017-08-09 15:08:17,219 INFO    : Workload     : |  period   100000 [us], duty_cycle  80 %
2017-08-09 15:08:17,220 INFO    : Workload     : |  run_time  80000 [us], sleep_time  20000 [us]
2017-08-09 15:08:17,221 INFO    : Workload     : ------------------------
2017-08-09 15:08:17,225 INFO    : Workload     : task [task1], sched: using default policy
2017-08-09 15:08:17,225 INFO    : Workload     :  | loops count: 1
2017-08

In [12]:
# Test execution of all these tasks into the LITTLE cluster
trace = rtapp.run(ftrace=te.ftrace, cgroup=cpuset_littles.name, out_dir=te.res_dir)

2017-08-09 15:08:28,332 INFO    : Workload     : Workload execution START:
2017-08-09 15:08:28,339 INFO    : Workload     :    CGMOUNT=/data/local/tmp/devlib-target/cgroups /data/local/tmp/bin/shutils cgroups_run_into /LITTLE /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/simple_00.json 2>&1


2017-08-09 15:08:46,919 INFO    : Workload     : Pulling trace file into [/media/build/workspace/lisa-github/results/20170809_150502/simple_00.dat]...


In [13]:
# Check tasks residency on little clsuter
trappy.plotter.plot_trace(trace)

In [14]:
# Compute and visualize tasks residencies on LITTLE clusterh CPUs
s = SchedMultiAssert(trappy.FTrace(trace), te.topology, execnames=tasks.keys())
residencies = s.getResidency('cluster', target.bl.littles, percent=True)
print json.dumps(residencies, indent=4)

{
    "16704": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "16697": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "16698": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "16699": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "16700": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "16701": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "16702": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "16703": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }
}


In [15]:
# Assert that ALL tasks have always executed only on LITTLE cluster
s.assertResidency('cluster', target.bl.littles,
                  99.9, operator.ge, percent=True, rank=len(residencies))

True

## Example of CPU controller usage

While the CPUSET is a controller to assign CPUs and memory nodes for a set of tasks, the CPU controller is used to assign CPU bandwidth.

In [16]:
# Get a reference to the CPU controller
cpu = target.cgroups.controller('cpu')

In [17]:
# Create a big partition on that CPUS
cpu_littles = cpu.cgroup('/LITTLE')

In [18]:
# Check the attributes available for this control group
print "LITTLE:\n", json.dumps(cpu_littles.get(), indent=4)

LITTLE:
{
    "rt_period_us": "1000000", 
    "shares": "1024", 
    "rt_runtime_us": "0"
}


In [19]:
# Set a 1CPU equivalent bandwidth for that CGroup
# cpu_littles.set(cfs_period_us=100000, cfs_quota_us=50000)
cpu_littles.set(shares=512)
print "LITTLE:\n", json.dumps(cpu_littles.get(), indent=4)

LITTLE:
{
    "rt_period_us": "1000000", 
    "shares": "512", 
    "rt_runtime_us": "0"
}


In [20]:
# Test execution of all these tasks into the LITTLE cluster
trace = rtapp.run(ftrace=te.ftrace, cgroup=cpu_littles.name)

2017-08-09 15:13:45,351 INFO    : Workload     : Workload execution START:
2017-08-09 15:13:45,352 INFO    : Workload     :    CGMOUNT=/data/local/tmp/devlib-target/cgroups /data/local/tmp/bin/shutils cgroups_run_into /LITTLE /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/simple_00.json 2>&1


2017-08-09 15:14:04,009 INFO    : Workload     : Pulling trace file into [.//simple_00.dat]...


In [21]:
# Check tasks residency on little cluster
trappy.plotter.plot_trace(trace)

## Example of CPUs isolation

In [22]:
# Isolate CPU0

# This works by moving all user-space tasks into a cpuset
# which does not include the specified list of CPUs to be
# isolated.
sandbox, isolated = target.cgroups.isolate(cpus=[0])

In [23]:
# Check the attributes available for the SANDBOX group
print "Sandbox:\n", json.dumps(sandbox.get(), indent=4)

Sandbox:
{
    "memory_pressure": "0", 
    "memory_spread_page": "0", 
    "notify_on_release": "0", 
    "sched_load_balance": "1", 
    "cpus": "1-7", 
    "effective_mems": "0", 
    "memory_spread_slab": "0", 
    "mem_hardwall": "0", 
    "cpu_exclusive": "0", 
    "mem_exclusive": "0", 
    "ls": " /data/local/tmp/devlib-target/cgroups/devlib_cgh5/DEVLIB_SBOX/cpuset.*", 
    "mems": "0", 
    "memory_migrate": "0", 
    "sched_relax_domain_level": "-1", 
    "effective_cpus": "1-7"
}


In [24]:
# Check the attributes available for the ISOLATED group
print "Isolated:\n", json.dumps(isolated.get(), indent=4)

Isolated:
{
    "memory_pressure": "0", 
    "memory_spread_page": "0", 
    "notify_on_release": "0", 
    "sched_load_balance": "1", 
    "cpus": "0", 
    "effective_mems": "0", 
    "memory_spread_slab": "0", 
    "mem_hardwall": "0", 
    "cpu_exclusive": "0", 
    "mem_exclusive": "0", 
    "ls": " /data/local/tmp/devlib-target/cgroups/devlib_cgh5/DEVLIB_ISOL/cpuset.*", 
    "mems": "0", 
    "memory_migrate": "0", 
    "sched_relax_domain_level": "-1", 
    "effective_cpus": "0"
}


In [25]:
# Run some workload, which is expected to not run in the ISOLATED cpus:
trace = rtapp.run(ftrace=te.ftrace)

2017-08-09 15:16:50,226 INFO    : Workload     : Workload execution START:
2017-08-09 15:16:50,229 INFO    : Workload     :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/simple_00.json 2>&1


2017-08-09 15:17:00,816 INFO    : Workload     : Pulling trace file into [.//simple_00.dat]...


In [26]:
# Check tasks was not running on ISOLATED CPUs
trappy.plotter.plot_trace(trace)

In [27]:
# Compute and visualize tasks residencies on ISOLATED CPUs
s = SchedMultiAssert(trappy.FTrace(trace), te.topology, execnames=tasks.keys())
residencies = s.getResidency('cpu', [0], percent=True)
print json.dumps(residencies, indent=4)

{
    "17120": {
        "residency": 0.0, 
        "task_name": "rt-app"
    }, 
    "17113": {
        "residency": 0.0, 
        "task_name": "rt-app"
    }, 
    "17114": {
        "residency": 0.0, 
        "task_name": "rt-app"
    }, 
    "17115": {
        "residency": 0.0, 
        "task_name": "rt-app"
    }, 
    "17116": {
        "residency": 0.0, 
        "task_name": "rt-app"
    }, 
    "17117": {
        "residency": 0.0, 
        "task_name": "rt-app"
    }, 
    "17118": {
        "residency": 0.0, 
        "task_name": "rt-app"
    }, 
    "17119": {
        "residency": 0.0, 
        "task_name": "rt-app"
    }
}


In [28]:
# Assert that ISOLATED CPUs was not running workload tasks
s.assertResidency('cpu', [0], 0.0, operator.eq, percent=True, rank=len(residencies))

True